# 1. Preprocessing Data

In [1]:
import pandas as pd
import re
import string
import os

from wordcloud import WordCloud
from pprint import pprint

import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk import flatten

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

import pyLDAvis.gensim_models

In [2]:
# Prefix Removal
def prefix_removal(content):
    colon = content.find(":")
    return content[colon+1:]

In [3]:
# Map to lowercase
def lowercase(content):
    return content.lower()

In [4]:
# Digit Removal
def digit_removal(content):
    return re.sub(r'\d+', '', content)

In [5]:
# Emoji Removal
def emoji_removal(content):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', content) # no emoji

In [6]:
# URL Removal
def url_removal(content):
    return re.sub(r'http\S+', '', content)

In [7]:
# Punctuation Removal by tokenizer
def punc_removal(content):
    punctuations = list(string.punctuation)
    punctuations.append('…')
    
    tk = TweetTokenizer(r'\w+')
    content_tokens = [i.strip("".join(punctuations)) for i in tk.tokenize(content) if i not in punctuations]
    return content_tokens

In [8]:
# Stop words removal
def stop_words_removal(content_tokens):
    stop_words = set(stopwords.words('english'))
#     stop_words.add('covid')
#     stop_words.add('coronaviru')
#     stop_words.add('coronavirus')
    return [w for w in content_tokens if not w in stop_words]

In [9]:
# Only keep english character
def non_english_removal(content_tokens):
    return [w for w in content_tokens if not re.findall("[^\u0000-\u05C0\u2100-\u214F]+", w) and len(w) > 2]

In [10]:
# Example of how to use the functions
print('\n')
print(">>>Example<<<\n")

content = "RT @DrLiMengYAN1: WHY ف_ج 你 and HOW the 2,999 ❤️😳 🈶️Lab-origin http://www.google.com of COVID-19 virus self-monitoring in a… "

print(">>>Original Content:", content, '\n')

content = lowercase(content)
print(">>>Lowercase:", content, '\n')

content = prefix_removal(content)
print(">>>Prefix Removal: ", content, '\n')

content = digit_removal(content)
print(">>>Digit Removal: ", content, '\n')

content = emoji_removal(content)
print(">>>Emoji Removal: ", content, '\n')

content = url_removal(content)
print(">>>Url Removal: ", content, '\n')

content_token = punc_removal(content)
print(">>>Punctuation Removal: ", content_token, '\n')

content_token = stop_words_removal(content_token)
print(">>>Stop Word Removal: ", content_token, '\n')

content_token = non_english_removal(content_token)
print(">>>Non English Character Removal: ", content_token, '\n')



>>>Example<<<

>>>Original Content: RT @DrLiMengYAN1: WHY ف_ج 你 and HOW the 2,999 ❤️😳 🈶️Lab-origin http://www.google.com of COVID-19 virus self-monitoring in a…  

>>>Lowercase: rt @drlimengyan1: why ف_ج 你 and how the 2,999 ❤️😳 🈶️lab-origin http://www.google.com of covid-19 virus self-monitoring in a…  

>>>Prefix Removal:   why ف_ج 你 and how the 2,999 ❤️😳 🈶️lab-origin http://www.google.com of covid-19 virus self-monitoring in a…  

>>>Digit Removal:   why ف_ج 你 and how the , ❤️😳 🈶️lab-origin http://www.google.com of covid- virus self-monitoring in a…  

>>>Emoji Removal:   why ف_ج  and how the ,  lab-origin http://www.google.com of covid- virus self-monitoring in a…  

>>>Url Removal:   why ف_ج  and how the ,  lab-origin  of covid- virus self-monitoring in a…  

>>>Punctuation Removal:  ['why', 'ف_ج', 'and', 'how', 'the', 'lab-origin', 'of', 'covid', 'virus', 'self-monitoring', 'in', 'a'] 

>>>Stop Word Removal:  ['ف_ج', 'lab-origin', 'covid', 'virus', 'self-monitoring'] 

>>>Non En

# 2. LDA

In [11]:
def find_file_name(file, country):
    for fname in os.listdir('Data/' + file + '/'):    # change directory as needed
        if country in fname:
            return 'Data/' + file + '/' + fname

In [12]:
def preprocess_tweet(content):
    content = lowercase(content)

    if content[:4] == 'RT @':
        content = prefix_removal(content)

    content = digit_removal(content)
    content = emoji_removal(content)
    content = url_removal(content)
    content_token = punc_removal(content)
    content_token = stop_words_removal(content_token)
    content_token = non_english_removal(content_token)
    return content_token

In [13]:
def read_file(directory):
    file = pd.read_csv(directory, index_col=False, on_bad_lines='skip', engine='python')
    
    print('diretory is', directory)
    twitters = []
    for i, t in enumerate(file['tweet']):
        content_token = preprocess_tweet(t)
        if len(content_token) >= 1:
            twitters.append(content_token)

        # Print examples
        if i < 2:
            print('line[{}] = {}'.format(i, t), '\n')
            
    if len(twitters) == 0:
        print("**********************************")
    return twitters


In [34]:
def find_tweets(country):
    root_dir = 'data/'
    early = find_file_name('Early_Processed', country)
    mid   = find_file_name('Mid_Processed', country)
    late  = find_file_name('Late_Processed', country)
    
    early_tweets = read_file(early)
    mid_tweets   = read_file(mid)
    late_tweets  = read_file(late)
    all_tweets = early_tweets + mid_tweets + late_tweets
    return [early_tweets, mid_tweets, late_tweets, all_tweets]

In [35]:
def LDAmodel(tweets, index=0, country='None'):
    names = ['Early', 'Mid', 'Late', 'All']
    print(len(tweets))
    id2word = corpora.Dictionary(tweets)

    # Create Corpus
    texts = tweets

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    print(i,len(corpus))


    # Build LDA model
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10)
    
    pyLDAvis.enable_notebook()
    vis_data = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    pyLDAvis.save_html(vis_data, 'RQ3 Visualization/' + names[index] + '/' + country + '.html')

In [37]:
countries = ['United States', 'United Kingdom', 'Thailand', 'India', 'Canada', 'Serbia', 'Australia', 'Nigeria', 'South Africa', 'Colombia']
for country in countries:
    results = find_tweets(country)
    for i, each in enumerate(results):
        LDAmodel(each, i, country)

diretory is Data/Early_Processed/8_United States Minor Outlying Islands.csv
line[0] = Spotting A Flying Praying Mantis Is Rare &amp; Hard Not To Notice ONCE SEEN! 4 The Past Ten Yrs My Training In Mixed Martial Arts Changed To Gi My Style &amp; Movements Within This Disipline To Take The Proper Stands &amp; Embraces For Your Love Ones Putting On A Gi For Our Life! #COVID19 https://t.co/9Q14PP9z3Y 

line[1] = Spotting A Flying Praying Mantis Is Rare &amp; Hard Not To Notice ONCE SEEN! 4 The Past Ten Yrs My Training In Mixed Martial Arts Changed To Gi My Style &amp; Movements Within This Disipline To Take The Proper Stands &amp; Embraces For Your Love Ones Putting On A Gi For Our Life! #COVID19 https://t.co/EX8fAACDGU 

diretory is Data/Mid_Processed/239919_United States_en.csv
line[0] = Sabotaging Trump was part of his plan all along.  #FireFauci 

line[1] = Apocalyptic forecast of 4,000 coronavirus deaths a day could be FIVE TIMES too high and had already been proved wrong when governm

/Users/caozhang/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


215567
1 215567


/Users/caozhang/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


29859
2 29859


...

/Users/caozhang/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


668
2 668


/Users/caozhang/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


8139
3 8139


/Users/caozhang/opt/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/caozhang/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/caozhang/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/caozhang/opt/anaconda3/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/caozhang/opt/anaconda3